# 人脸识别系统

In [1]:
import tensorflow as tf
gpu = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpu[0], True)

In [2]:
import os
import time
import numpy as np
import matplotlib.pyplot as plt
import cv2
import tqdm  # 进度条工具

## 人脸数据采集

In [3]:
num = 200  # 设置拍摄的照片数量
img_dir = '../data/faces'  # 设置照片的存储位置
name = 'linjing'
time_delay = 0
wait_key = 200  # 展示拍摄图片时取消注释

In [4]:
name_path = os.path.join(img_dir, name)
if not os.path.exists(name_path):  # 创建文件夹
    os.makedirs(name_path)
camera = cv2.VideoCapture(0)    # 调用摄像头
for i in tqdm.tqdm(range(num)):
    success, img = camera.read()  # 从摄像头读取照片
#     time.sleep(time_delay)
    if success:
        img_name = str(i) + '.jpg'    
        cv2.imshow(img_name, img)  # 展示拍摄图片
        cv2.waitKey(wait_key)
        cv2.destroyAllWindows()
        cv2.imwrite(os.path.join(name_path, img_name), img)
camera.release()  # 当一切都完成后，关闭摄像头并释放所占资源

100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [01:11<00:00,  2.78it/s]


## 将人脸信息从图片中筛选出来

In [5]:
faceCascade = cv2.CascadeClassifier('../models/haarcascade_frontalface_default.xml')
faceCascade.load(r'../models/haarcascade_frontalface_default.xml')

True

In [6]:
gray_img = cv2.imread('../data/faces/linjing/10.jpg', 0)

In [7]:
# 调用函数detectMultiScale
faces = faceCascade.detectMultiScale(gray_img)

In [ ]:
for (x, y, w, h) in faces:
    cv2.rectangle(gray_img, (x, y), (x + h, y + w), (0, 0, 255), 2)
plt.imshow(gray_img, cmap='gray')
plt.axis('off')
plt.show()

In [ ]:
for (x, y, w, h) in faces:
    face = gray_img[y:y+h, x:x+w]
plt.imshow(face, cmap='gray')
plt.axis('off')
plt.show()

## 数据的预处理

In [10]:
faceCascade = cv2.CascadeClassifier('../models/haarcascade_frontalface_default.xml')
faceCascade.load(r'../models/haarcascade_frontalface_default.xml')

True

In [11]:
peoples = os.listdir('../data/faces/')
for people in peoples:
    img_names = os.listdir('../data/faces/' + people)
    for img_name in img_names:
        img_path = '../data/faces/' + people + '/' + img_name
        img = cv2.imread(img_path, 0)
        # 检测人脸
        faces = faceCascade.detectMultiScale(img)
        # 获取人脸部分
        for (x, y, w, h) in faces:
            face = img[y:y + h, x:x + w]
        face = cv2.resize(face, (64, 64))
        if not os.path.exists('../data/detect/' + people):
            os.mkdir('../data/detect/' + people)
        cv2.imwrite('../data/detect/' + people + '/' + img_name, face)

## 读取处理好的数据

In [12]:
path = '../data/detect/'
names = os.listdir(path)
names_dic = {i:j for i, j in enumerate(names)}
imgs, labels = [], []
for i in range(len(names)):
    img_names = os.listdir(path + names[i])
    for j in range(len(img_names)):
        labels.append(i)
        img = cv2.imread(path + names[i] + '/' + img_names[j], 0)
        imgs.append(img)

In [13]:
names_dic

{0: 'linjing'}

In [14]:
labels = np.array(labels)
imgs = np.array(imgs)

In [15]:
imgs = np.expand_dims(imgs, -1)
imgs.shape

(200, 64, 64, 1)

In [16]:
labels.shape

(200,)

## 切分训练集和测试集

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(imgs, labels, test_size=0.2)

## 模型的搭建

In [18]:
model = tf.keras.Sequential(name='LeNet5')

- filters: 卷积核的个数
- kernel_size: 卷积核的大小
- striders: 卷积核移动的步长
- input_shape: 输入到模型中的数据的尺寸，这个只在网络的第一层中指定
- activation: 非线性激活函数

In [19]:
model.add(tf.keras.layers.Conv2D(filters=6, kernel_size=(5, 5), strides=(1, 1), input_shape=(64, 64, 1), activation='relu'))
model.add(tf.keras.layers.MaxPool2D()) # 最大池化层
model.add(tf.keras.layers.Conv2D(filters=16, kernel_size=(5, 5), strides=(1, 1), activation='relu')) # 卷积层
model.add(tf.keras.layers.MaxPool2D()) # 最大池化层
model.add(tf.keras.layers.Flatten()) # 展平层

In [20]:
model.add(tf.keras.layers.Dense(units=120, activation='relu')) # units: 全连接层的神经元数量
model.add(tf.keras.layers.Dense(units=84, activation='relu'))
model.add(tf.keras.layers.Dense(units=len(names_dic), activation='softmax'))

In [21]:
model.summary()

Model: "LeNet5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 60, 60, 6)         156       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 30, 30, 6)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 26, 26, 16)        2416      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 16)        0         
_________________________________________________________________
flatten (Flatten)            (None, 2704)              0         
_________________________________________________________________
dense (Dense)                (None, 120)               324600    
_________________________________________________________________
dense_1 (Dense)              (None, 84)                10164

## 模型的编译和训练

In [22]:
model.compile(optimizer='adam', loss=tf.keras.losses.sparse_categorical_crossentropy, metrics=tf.keras.metrics.sparse_categorical_accuracy)

In [23]:
model.fit(X_train, y_train, epochs=10, batch_size=16, validation_split=0.2)

Epoch 1/10
8/8 [==============================] - 0s 26ms/step - loss: 0.0000e+00 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.0000e+00 - val_sparse_categorical_accuracy: 1.0000
Epoch 2/10
8/8 [==============================] - 0s 7ms/step - loss: 0.0000e+00 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.0000e+00 - val_sparse_categorical_accuracy: 1.0000
Epoch 3/10
8/8 [==============================] - 0s 7ms/step - loss: 0.0000e+00 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.0000e+00 - val_sparse_categorical_accuracy: 1.0000
Epoch 4/10
8/8 [==============================] - 0s 7ms/step - loss: 0.0000e+00 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.0000e+00 - val_sparse_categorical_accuracy: 1.0000
Epoch 5/10
8/8 [==============================] - 0s 7ms/step - loss: 0.0000e+00 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.0000e+00 - val_sparse_categorical_accuracy: 1.0000
Epoch 6/10
8/8 [==============================] - 0s 7ms/step - loss: 0.0000e+0

## 模型验证

In [24]:
model.evaluate(X_test, y_test)

2/2 [==============================] - 0s 32ms/step - loss: 0.0000e+00 - sparse_categorical_accuracy: 1.0000


[0.0, 1.0]

## 模型的测试

In [25]:
# 调用摄像头拍摄照片
camera = cv2.VideoCapture(0)    # 调用摄像头
success, img = camera.read()  # 从摄像头读取照片
camera.release()  # 当一切都完成后，关闭摄像头并释放所占资源

In [ ]:
plt.imshow(img[:,:,::-1])
plt.show()

In [27]:
# 将图像转化为灰度图
gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [ ]:
plt.imshow(gray_img, cmap='gray')
plt.show()

In [29]:
# 调用函数detectMultiScale
faces = faceCascade.detectMultiScale(gray_img)

In [ ]:
for (x, y, w, h) in faces:
    face = gray_img[y:y + h, x:x + w]
face = cv2.resize(face, (64, 64))
plt.imshow(face, cmap='gray')
plt.axis('off')
plt.show()

In [31]:
face = np.expand_dims(face, 0)
face = np.expand_dims(face, -1)
face.shape

(1, 64, 64, 1)

In [32]:
index_pred = model.predict(face).argmax(axis=1)

In [33]:
# 利用姓名字典还原真实姓名
names_dic[index_pred[0]]

'linjing'